In [ ]:
#FANZAの情報スクレイピング　現在

import requests
from bs4 import BeautifulSoup
import openpyxl
import os
import tkinter as tk
from tkinter import filedialog, messagebox
import sys

def download_images(url, excel_path):
    # ヘッダーとクッキーの設定
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
    }
    cookie = {'age_check_done': '1'}

    # 固定のURLリストファイルパス
    url_list_path = 
    # セッションの初期化
    session = requests.Session()
    session.get(url, headers=headers, cookies=cookie)

    # Excelファイルのロード
    try:
        wb = openpyxl.load_workbook(excel_path)
        sheet = wb['記事作成部分']
    except Exception as e:
        messagebox.showerror("エラー", f"Excelファイルの読み込みに失敗しました: {e}")
        return

    row_num = 4  # 開始行
    try:
        with open(url_list_path, 'r', encoding='UTF-8') as f:
            urls = f.readlines()
    except Exception as e:
        messagebox.showerror("エラー", f"固定URLリストファイルの読み込みに失敗しました: {e}")
        return

    for idx, line in enumerate(reversed(urls), start=1):
        page_url = line.strip()
        print(f"Processing URL ({idx}): {page_url}")

        try:
            # HTMLの取得と解析
            response = session.get(page_url, headers=headers, cookies=cookie)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # データの抽出
            url_tag = soup.find('meta', property="og:url")
            url_title = soup.find(id='title')
            name = soup.find(id='performer')
            time_tag = soup.find_all('td')

            # 要素の存在確認とデータ取得
            url_tag_content = url_tag['content'] if url_tag else "URLが見つかりません"
            url_title_text = url_title.text.strip() if url_title else "タイトルが見つかりません"
            name_text = name.text.strip() if name else "出演者情報なし"
            time_text = time_tag[10].text.strip()[:-1] if len(time_tag) > 10 else "時間情報なし"

            # Excelに書き込み
            sheet.cell(row=row_num, column=3, value=url_title_text)
            sheet.cell(row=row_num, column=4, value=time_text)
            sheet.cell(row=row_num, column=5, value=url_tag_content)
            sheet.cell(row=row_num, column=6, value=name_text)

            row_num += 1

        except Exception as e:
            print(f"URL {page_url} の処理中にエラーが発生しました: {e}")
            continue

    # Excelファイルを保存
    try:
        wb.save(excel_path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {excel_path}")
    except Exception as e:
        messagebox.showerror("エラー", f"Excelファイルの保存に失敗しました: {e}")

def select_file_and_run(root):
    # Excelファイル選択
    excel_path = filedialog.askopenfilename(
        title="Excelファイルを選択してください",
        filetypes=[("Excel Files", "*.xlsx;*.xls")],
        initialdir=os.path.expanduser("~/Documents")
    )

    if not excel_path:
        messagebox.showwarning("警告", "Excelファイルが選択されていません")
        return

    # 確認メッセージ
    if not messagebox.askyesno("確認", f"以下のExcelファイルで処理を開始しますか？\n\nExcelファイル: {excel_path}\n固定URLリスト: C:\\Users\\yuruy\\Downloads\\AV\\FANZA_URL.txt"):
        return

    # GUIを閉じる
    root.destroy()

    # ダウンロード処理を実行
    try:
        url_certification = 'https://www.dmm.co.jp/age_check/=/declared=yes/?rurl=https%3A%2F%2Fwww.dmm.co.jp%2Ftop%2F'
        download_images(url_certification, excel_path)
        sys.exit()
    except Exception as e:
        messagebox.showerror("エラー", f"処理中にエラーが発生しました:\n{e}")
        sys.exit()

def main():
    # Tkinterウィンドウを設定
    root = tk.Tk()
    root.title("FANZA情報スクレイピング")

    # ウィンドウを常に最前面に設定
    root.attributes("-topmost", True)

    # ウィンドウのサイズを設定
    root.geometry("400x300")

    # 説明ラベル
    label = tk.Label(root, text="Excelファイルを選択してください", font=("Arial", 12))
    label.pack(pady=20)

    # 実行ボタン
    button = tk.Button(root, text="ファイルを選択して実行", command=lambda: select_file_and_run(root), font=("Arial", 12), bg="lightblue")
    button.pack(pady=40)

    # Tkinterメインループ開始
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
#FANZAの画像スクレイピング　現在

import os
import requests
from bs4 import BeautifulSoup
import openpyxl
import tkinter as tk
from tkinter import filedialog, messagebox
import sys

def download_images(url, excel_path):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    cookie = {'age_check_done': '1'}
    session = requests.session()
    session.get(url)

    # Excelファイルからフォルダ名を取得
    wb = openpyxl.load_workbook(excel_path)
    sheet = wb.active  # アクティブシートを取得
    folder_name = sheet["C2"].value  # C2セルの値を取得
    folder_directory = os.path.join(r"", folder_name) #フォルダのパスを生成

    # フォルダ作成
    os.makedirs(folder_directory, exist_ok=True)
    for i in range(1, 21):  # 1から20の番号でサブフォルダを作成
        subfolder_name = str(i).zfill(2)  # "01", "02", ..., "20" の形式
        os.makedirs(os.path.join(folder_directory, subfolder_name), exist_ok=True)

    with open(r"", 'r', encoding='UTF-8') as f: #URLリストファイルのパス
        lines = f.readlines()
        
        for idx, line in enumerate(reversed(lines), start=1):
            soup = BeautifulSoup(session.get(line.strip(), headers=headers, cookies=cookie).content, 'html.parser')
            package_image = soup.find('div', class_='center').find('a').get("href")
            package_image_name = package_image.split('/')[-1]
            print(f"Downloading package image: {package_image}")

            # フォルダ番号を2桁の形式に変換
            subfolder = os.path.join(folder_directory, str((idx - 1) % 20 + 1).zfill(2))  
            r = requests.get(package_image)
            with open(os.path.join(subfolder, package_image_name), 'wb') as img_file:
                img_file.write(r.content)

            try:
                sample_image_list = soup.find('div', class_='d-zoomimg-sm').find_all('a')
                for sample_image in sample_image_list:
                    sample_image_url = sample_image.get("href")
                    sample_image_name = sample_image_url.split('/')[-1]
                    r = requests.get(sample_image_url)
                    with open(os.path.join(subfolder, sample_image_name), 'wb') as img_file:
                        img_file.write(r.content)
            except Exception as e:
                print(f"Error downloading sample images for URL {line.strip()}: {e}")

def select_excel_and_run(root):
    # ファイル選択ダイアログを開く
    file_path = filedialog.askopenfilename(
        title="Excelファイルを選択してください",
        filetypes=[("Excel Files", "*.xlsx;*.xls")],
        initialdir=os.path.expanduser("~/Documents")  # 初期ディレクトリをDownloadsに設定
    )

    if not file_path:
        messagebox.showwarning("警告", "ファイルが選択されていません")
        return

    # 確認メッセージ
    if not messagebox.askyesno("確認", f"選択したファイル: {file_path}\nこのファイルで処理を開始しますか？"):
        return

    # GUIを閉じる
    root.destroy()

    try:
        # ダウンロード処理を実行
        url_certification = 'https://www.dmm.co.jp/age_check/=/declared=yes/?rurl=https%3A%2F%2Fwww.dmm.co.jp%2Ftop%2F'
        download_images(url_certification, file_path)

        # 処理終了メッセージを表示
        messagebox.showinfo("完了", "ダウンロードが完了しました")

        # プログラムを終了
        sys.exit()
    except Exception as e:
        messagebox.showerror("エラー", f"処理中にエラーが発生しました:\n{e}")
        sys.exit()

def main():
    # Tkinterウィンドウを設定
    root = tk.Tk()
    root.title("Excelファイル選択")

    # ウィンドウを常に最前面に設定
    root.attributes("-topmost", True)

    # ウィンドウのサイズを設定
    root.geometry("400x200")

    # 説明ラベル
    label = tk.Label(root, text="処理に使用するExcelファイルを選択してください", font=("Arial", 12))
    label.pack(pady=20)

    # 実行ボタン
    button = tk.Button(root, text="Excelファイルを選択", command=lambda: select_excel_and_run(root), font=("Arial", 12), bg="lightblue")
    button.pack(pady=20)

    # Tkinterメインループ開始
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
#勉強中のコード

import requests
from bs4 import BeautifulSoup
import openpyxl
import os
import tkinter as tk
from tkinter import filedialog, messagebox
import sys
import re

def extract_runtime(soup):
    """
    様々な形式の再生時間情報（例："1時間20分", "20分", "01:20:00"）に対応して抽出する関数。
    該当する<td>タグやテキストがあれば、合計分数を文字列で返す。
    """
    # パターン1: "1時間20分"または"20分"形式
    time_tag = soup.find(lambda tag: tag.name == "td" and re.search(r'(\d+\s*時間\s*)?\d+\s*分', tag.text))
    if time_tag:
        raw_time = time_tag.text.strip()
        hm_match = re.search(r'(?:(\d+)\s*時間)?\s*(\d+)\s*分', raw_time)
        if hm_match:
            hours = hm_match.group(1)
            minutes = hm_match.group(2)
            total_minutes = (int(hours) * 60 if hours else 0) + int(minutes)
            return str(total_minutes)
    
    # パターン2: "hh:mm(:ss)"形式
    time_tag = soup.find(lambda tag: tag.name == "td" and re.search(r'\d+:\d+(?::\d+)?', tag.text))
    if time_tag:
        raw_time = time_tag.text.strip()
        time_match = re.search(r'(\d+):(\d+)(?::(\d+))?', raw_time)
        if time_match:
            hours, minutes, _ = time_match.groups()
            total_minutes = int(hours) * 60 + int(minutes)
            return str(total_minutes)
    
    return "時間情報なし"

def download_data(url, excel_path):
    # ヘッダーとクッキーの設定
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
    }
    cookie = {'age_check_done': '1'}

    # 固定のURLリストファイルパス
    url_list_path = 

    # セッションの初期化
    session = requests.Session()
    session.get(url, headers=headers, cookies=cookie)

    # Excelファイルのロード
    try:
        wb = openpyxl.load_workbook(excel_path)
        sheet = wb['記事作成部分']
    except Exception as e:
        messagebox.showerror("エラー", f"Excelファイルの読み込みに失敗しました: {e}")
        return

    row_num = 4  # 書き込み開始行

    try:
        with open(url_list_path, 'r', encoding='UTF-8') as f:
            urls = f.readlines()
    except Exception as e:
        messagebox.showerror("エラー", f"固定URLリストファイルの読み込みに失敗しました: {e}")
        return

    for idx, line in enumerate(reversed(urls), start=1):
        page_url = line.strip()
        print(f"Processing URL ({idx}): {page_url}")

        try:
            # HTMLの取得と解析
            response = session.get(page_url, headers=headers, cookies=cookie)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # データの抽出
            url_tag = soup.find('meta', property="og:url")
            url_title = soup.find(id='title')
            performer = soup.find(id='performer')
            runtime = extract_runtime(soup)

            url_tag_content = url_tag['content'] if url_tag else "URLが見つかりません"
            url_title_text = url_title.text.strip() if url_title else "タイトルが見つかりません"
            performer_text = performer.text.strip() if performer else "出演者情報なし"

            # Excelに書き込み
            sheet.cell(row=row_num, column=3, value=url_title_text)
            sheet.cell(row=row_num, column=4, value=runtime)
            sheet.cell(row=row_num, column=5, value=url_tag_content)
            sheet.cell(row=row_num, column=6, value=performer_text)

            row_num += 1

        except Exception as e:
            print(f"URL {page_url} の処理中にエラーが発生しました: {e}")
            continue

    # Excelファイルを保存
    try:
        wb.save(excel_path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {excel_path}")
    except Exception as e:
        messagebox.showerror("エラー", f"Excelファイルの保存に失敗しました: {e}")

def select_file_and_run(root):
    # Excelファイル選択
    excel_path = filedialog.askopenfilename(
        title="Excelファイルを選択してください",
        filetypes=[("Excel Files", "*.xlsx;*.xls")],
        initialdir=os.path.expanduser("~/Documents")
    )

    if not excel_path:
        messagebox.showwarning("警告", "Excelファイルが選択されていません")
        return

    # 確認メッセージ
    confirm_msg = (
        f"以下のExcelファイルで処理を開始しますか？\n\n"
        f"Excelファイル: {excel_path}\n"
        f"固定URLリスト: C:\\Users\\yuruy\\Downloads\\AV\\FANZA_URL.txt"
    )
    if not messagebox.askyesno("確認", confirm_msg):
        return

    # GUIを閉じる
    root.destroy()

    try:
        url_certification = 'https://www.dmm.co.jp/age_check/=/declared=yes/?rurl=https%3A%2F%2Fwww.dmm.co.jp%2Ftop%2F'
        download_data(url_certification, excel_path)
        sys.exit()
    except Exception as e:
        messagebox.showerror("エラー", f"処理中にエラーが発生しました:\n{e}")
        sys.exit()

def main():
    root = tk.Tk()
    root.title("FANZA情報スクレイピング")
    root.attributes("-topmost", True)
    root.geometry("400x300")

    label = tk.Label(root, text="Excelファイルを選択してください", font=("Arial", 12))
    label.pack(pady=20)

    button = tk.Button(root, text="ファイルを選択して実行", command=lambda: select_file_and_run(root), font=("Arial", 12), bg="lightblue")
    button.pack(pady=40)

    root.mainloop()

if __name__ == "__main__":
    main()


Processing URL (1): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=sivr00157/?i3_ref=list&i3_ord=12&i3_pst=1
Processing URL (2): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=13dsvr01219/?i3_ref=search&i3_ord=31&i3_pst=1
Processing URL (3): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=ebvr00102/?i3_ref=search&i3_ord=32&i3_pst=1
Processing URL (4): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=sivr00127/?i3_ref=search&i3_ord=18&i3_pst=1
Processing URL (5): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=savr00180/?i3_ref=search&i3_ord=16&i3_pst=1
Processing URL (6): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=sivr00313/?i3_ref=search&i3_ord=15&i3_pst=1
Processing URL (7): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=h_1456pxvr00037/?i3_ref=search&i3_ord=14&i3_pst=1
Processing URL (8): https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=sivr00341/?i3_ref=search&i3_ord=10&i3_pst=1
Processing URL (9): https://www.dmm.co.jp/digital/videoa/-/detail/

SystemExit: 

C:\Users\yuruy\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
